In [1]:
import pandas as pd
from funciones import month_year_iter

In [2]:
rutaR=r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes"

cols=["Centro","Descripción Centro", "Receta","Desc. Receta","Clase","Material","Texto breve material",
      "Unidad medida base (=EINHEIT)","Cantidad tomada (EINHEIT)","Costo Tomado"]

conv={"Receta":str,"Material":str}

mesA=4
yearA=2023
mesI=5
yearI=2023

In [3]:
for tiempo in month_year_iter(mesA,yearA,mesI,yearI):
    df = pd.read_excel(rutaR+"\Consumos\{}\{}. Consumos.xlsx".format(tiempo[0],tiempo[1]),usecols=cols,converters=conv)
    df=df[~(df["Receta"]=="No encontrado")]
    df=df.groupby(cols[:-2]).sum().reset_index()
    
    df["Tipo Material"]=df["Material"].str[0].replace({"5":"PT","3":"MP","4":"EP","6":"ST","P":"CONV","2":"SP"})
    df=df.rename(columns={"Cantidad tomada (EINHEIT)":"Cantidad Real","Costo Tomado":"Costo Real","Unidad medida base (=EINHEIT)":"UM"})
    dfPT=df.loc[df["Clase"]=="Cabecera",["Centro","Descripción Centro","Receta","Cantidad Real"]].copy().rename(columns={"Cantidad Real":"Cantidad Entregada"})
    df=df.merge(dfPT,on=["Centro","Descripción Centro","Receta"],how="left")
    del dfPT
    df["Factor de uso"]=df["Cantidad Real"].divide(-df["Cantidad Entregada"])
    df=df[~df["Cantidad Entregada"].isna()]
    df=df[df["Cantidad Real"]!=0.0]
    df["Costo Unitario"]=df["Costo Real"].divide(df["Cantidad Real"])
    #del df["Cantidad Real"]
    del df["Costo Real"]
    del df["Cantidad Entregada"]
    aux=df.copy()
    iteraciones=10
    for i in range(1,iteraciones):
        if i == 1:
            df=df.rename(columns=dict(zip(df.columns, [j+"_{}".format(1) for j in df.columns])))
        cabs=df[df["Clase_{}".format(i)]=="Cabecera"]
        df=df[df["Clase_{}".format(i)]!="Cabecera"]
        temp=aux.copy().rename(columns=dict(zip(aux.columns, [j+"_{}".format(i+1) for j in aux.columns])))
        temp=temp[temp["Clase_{}".format(i+1)]!="Cabecera"]
        df=df.merge(temp,how="left",
                 left_on=["Centro_{}".format(i),"Descripción Centro_{}".format(i),"Material_{}".format(i)],
                    right_on=["Centro_{}".format(i+1),"Descripción Centro_{}".format(i+1),"Receta_{}".format(i+1)])
        df=pd.concat([cabs,df])
        for c in aux.columns:
            if c != "Factor de uso":
                df[c+"_{}".format(i+1)].fillna(df[c+"_{}".format(i)],inplace=True)
            else:
                df[c+"_{}".format(i+1)].fillna(1,inplace=True)

    df["Factor de uso"]=1
    for i in range(1,iteraciones+1):
        df["Factor de uso"]=df["Factor de uso"]*df["Factor de uso_{}".format(i)]

    df["Costo Unitario Calculado"]=df["Factor de uso"]*df["Costo Unitario_{}".format(iteraciones)]
    df["Costo Unitario Estandar"]=0
    df.loc[df["Clase_{}".format(iteraciones)]=="Cabecera","Costo Unitario Calculado"]=0
    df.loc[df["Clase_{}".format(iteraciones)]=="Cabecera","Costo Unitario Estandar"]=df.loc[df["Clase_{}".format(iteraciones)]=="Cabecera","Costo Unitario_{}".format(iteraciones)]

    df["Cantidad Entregada"]=0
    df.loc[df["Clase_{}".format(iteraciones)]=="Cabecera","Cantidad Entregada"]=df.loc[df["Clase_{}".format(iteraciones)]=="Cabecera","Cantidad Real_{}".format(iteraciones)]
    
    df.to_excel(rutaR+"\Costo Real\{}\{}. Costo Real v2.xlsx".format(tiempo[0],tiempo[1]),index=None)
    print(tiempo)

(2023, 4)
